In [1]:
from pathlib import Path
from image_modalities_classifier.models.trainer import ModalityModelTrainer

/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
artifacts_folder =  Path.home() / "Documents/modality_classifiers" 

dataset_filepath = artifacts_folder / "data/updated_cord19/cord19_microscopy_v1.parquet"
num_classes = 3
base_img_dir = "/media/cumulus/curation_data"
output_dir = artifacts_folder / "models"
taxonomy = "cord19"
classifier_name = "microscopy"
project = "biocuration"
num_workers = 12
name = "resnet101"
lr = 1e-3

In [3]:
trainer = ModalityModelTrainer(
    dataset_filepath, base_img_dir, output_dir, taxonomy, classifier_name, project, num_workers=num_workers, model_name=name, learning_rate=lr)
trainer.run()

Global seed set to 443
Global seed set to 443
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtrell2. Use `wandb login --relogin` to force relogin


/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /home/jtt/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:01<00:00, 117MB/s] 
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jtt/repos/bio-search/image-modalities-classifier/.venv/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/jtt/Documents/modality_classifiers/models/cord19/microscopy exists and is not empty.
  rank_zero_warn(f"Checkpoint directory 

Epoch 0: 100%|██████████| 347/347 [01:59<00:00,  2.89it/s, loss=0.151, v_num=c407]

Metric val_loss improved. New best score: 0.587


Epoch 1: 100%|██████████| 347/347 [01:59<00:00,  2.90it/s, loss=0.0933, v_num=c407]

...

Epoch 15: 100%|██████████| 347/347 [01:58<00:00,  2.92it/s, loss=0.0439, v_num=c407]

Monitored metric val_loss did not improve in the last 5 records. Best score: 0.335. Signaling Trainer to stop.


Epoch 15: 100%|██████████| 347/347 [01:58<00:00,  2.92it/s, loss=0.0439, v_num=c407]


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
lr-Adam,████████▂▂▂▂▂▂▂▁
train_acc,▁▃▅▅▅▅▅▆▆▆▇▇▇███
train_loss,▅▅▄▂▂▆▇▂▃▁▂▂▅▂▁▂▆▂▂▁▃▃▂▂▂▂▁▃▂▁▂▇▂█▁▄▃▄▁▃
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▄▅▆▆▆▇▆▇▇███▇▇▇
val_loss,█▅▄▂▃▃▃▃▁▂▁▁▁▄▂▂
epoch,15
lr-Adam,1e-05
train_acc,0.98406
train_loss,0.06164


'resnet101_microscopy_2.pt'

In [4]:
import torch

torch.cuda.is_available()

True

In [4]:
trainer.model.free()

AttributeError: 'ModalityModelTrainer' object has no attribute 'model'

: 

In [4]:
import pandas as pd
df = pd.read_parquet('/home/jtt/Documents/modality_classifiers/data/cord19/cord19_experimental_v1.parquet')
df.head()

,img,label,source,img_path,caption,split_set,features,prediction,width,height,pred_probs,ms_metric,en_metric
0,1423-0127-16-6-5-2.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1423-...,Cleavage specificities of human EndoG variants...,TRAIN,"[0.30319658, 0.9711439, 0.6038045, 0.39786372,...",exp.gel,569,252,"[0.9944015145301819, 0.00559849850833416]",NaN,NaN
1,1423-0127-16-6-5-3.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1423-...,Cleavage specificities of human EndoG variants...,TRAIN,"[0.85481423, 1.3120571, 0.4690486, 0.40002385,...",exp.gel,569,193,"[0.9987044334411621, 0.0012955853017047048]",NaN,NaN
2,1423-0127-17-22-2-1.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1423-...,MBP-pulldown and mammalian two hybrid assay in...,TRAIN,"[0.5675413, 0.3917636, 0.18453613, 1.0121593, ...",exp.gel,298,263,"[0.9999366998672485, 6.327618757495657e-05]",NaN,NaN
3,1465-9921-10-26-1-2.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1465-...,CFTR expression in bone marrow derived dendrit...,TRAIN,"[0.16412717, 0.56114954, 0.22790892, 1.623608,...",exp.gel,542,174,"[0.9999823570251465, 1.763599539117422e-05]",NaN,NaN
4,1465-9921-10-26-1-3.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1465-...,CFTR expression in bone marrow derived dendrit...,TRAIN,"[0.5535225, 0.35870844, 1.3544716, 1.871606, 0...",exp.gel,542,248,"[0.9988740086555481, 0.0011259742313995957]",NaN,NaN


In [6]:
df2 = df[df.split_set != "UNL"]

In [7]:
df2.head()

,img,label,source,img_path,caption,split_set,features,prediction,width,height,pred_probs,ms_metric,en_metric
0,1423-0127-16-6-5-2.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1423-...,Cleavage specificities of human EndoG variants...,TRAIN,"[0.30319658, 0.9711439, 0.6038045, 0.39786372,...",exp.gel,569,252,"[0.9944015145301819, 0.00559849850833416]",NaN,NaN
1,1423-0127-16-6-5-3.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1423-...,Cleavage specificities of human EndoG variants...,TRAIN,"[0.85481423, 1.3120571, 0.4690486, 0.40002385,...",exp.gel,569,193,"[0.9987044334411621, 0.0012955853017047048]",NaN,NaN
2,1423-0127-17-22-2-1.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1423-...,MBP-pulldown and mammalian two hybrid assay in...,TRAIN,"[0.5675413, 0.3917636, 0.18453613, 1.0121593, ...",exp.gel,298,263,"[0.9999366998672485, 6.327618757495657e-05]",NaN,NaN
3,1465-9921-10-26-1-2.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1465-...,CFTR expression in bone marrow derived dendrit...,TRAIN,"[0.16412717, 0.56114954, 0.22790892, 1.623608,...",exp.gel,542,174,"[0.9999823570251465, 1.763599539117422e-05]",NaN,NaN
4,1465-9921-10-26-1-3.jpg,exp.gel,clef16,subfigure-classification/2016/train/GGEL/1465-...,CFTR expression in bone marrow derived dendrit...,TRAIN,"[0.5535225, 0.35870844, 1.3544716, 1.871606, 0...",exp.gel,542,248,"[0.9988740086555481, 0.0011259742313995957]",NaN,NaN


In [10]:
df2[df2.label.isna()]

,img,label,source,img_path,caption,split_set,features,prediction,width,height,pred_probs,ms_metric,en_metric
16326,PMC1074749gni0647_1001,None,cord19,cord19-uic/PMC1074749/gni064/7_1/001.jpg,Figure 2. Autoradiogram of 32P-labelled double...,TRAIN,"[0.11695894, 1.4279399, 0.19408609, 1.0856833,...",exp.gel,1015,1167,"[0.9998939037322998, 0.0001061404327629134]",0.999788,0.001077
16327,PMC1074749gni0647_1002,None,cord19,cord19-uic/PMC1074749/gni064/7_1/002.jpg,Figure 2. Autoradiogram of 32P-labelled double...,TRAIN,"[0.08851135, 1.7101694, 0.40700686, 1.3455511,...",exp.gel,1026,1167,"[0.9999943971633911, 5.564953880821122e-06]",0.999989,0.000073
16331,PMC1074749gni0647_1,None,cord19,cord19-uic/PMC1074749/gni064/7_1.jpg,Figure 2. Autoradiogram of 32P-labelled double...,TRAIN,"[0.19898227, 2.1635885, 0.37557727, 0.68559366...",exp.gel,2125,1167,"[0.9999864101409912, 1.3544691682909615e-05]",0.999973,0.000165
16334,PMC11705820023923_1003,None,cord19,cord19-uic/PMC1170582/002392/3_1/003.jpg,Fig. 1. An in vitro system to study RNA recomb...,TRAIN,"[0.72941417, 1.0166347, 1.0784396, 1.9412333, ...",exp.gel,634,541,"[0.9998990297317505, 0.00010101350926561281]",0.999798,0.001030
16335,PMC11705820023924_1003,None,cord19,cord19-uic/PMC1170582/002392/4_1/003.jpg,Fig. 2. The motif1-hairpin facilitates 3Ð-TX....,TRAIN,"[0.6043587, 0.7361291, 1.3665982, 1.9135891, 1...",exp.gel,433,426,"[0.9998519420623779, 0.00014799261407461017]",0.999704,0.001453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28746,PMC7154305CAVP_36_2155385_2004,None,cord19,cord19-uic/PMC7154305/CAVP_36_215538/5_2/004.jpg,Figure 4. Western blot analysis of the amount ...,TRAIN,"[0.34352347, 0.63927853, 0.8144694, 0.8321744,...",exp.gel,852,95,"[0.9997183680534363, 0.00028156282496638596]",0.999437,0.002583
28748,PMC7153726main10_1001,None,cord19,cord19-uic/PMC7153726/main/10_1/001.jpg,Figure 5 Bronchiolitis obliterans (BOS) and ai...,TRAIN,"[1.0273337, 2.872028, 0.01068689, 0.0, 0.0, 5....",exp.pla,2036,1303,"[0.00022263296705204993, 0.9997773766517639]",0.999555,0.002095
28749,PMC7153726main10_1,None,cord19,cord19-uic/PMC7153726/main/10_1.jpg,Figure 5 Bronchiolitis obliterans (BOS) and ai...,TRAIN,"[1.0274416, 2.8717542, 0.010641738, 0.0, 0.0, ...",exp.pla,2036,1303,"[0.00022337204427458346, 0.9997766613960266]",0.999553,0.002101
28750,PMC7154514supplementaryfigsplit_mmc13_1005,None,cord19,cord19-uic/PMC7154514/supplementary/figsplit_m...,,TRAIN,"[1.0600636, 0.12502278, 0.35921696, 1.2463709,...",exp.gel,1654,32,"[0.9999011754989624, 9.877476986730471e-05]",0.999802,0.001010
